In [0]:
from pyspark.sql.functions import col, explode, expr, create_map
import json
import requests

query = "https://api.coindesk.com/v1/bpi/currentprice.json"
response = requests.get(query)
response.raise_for_status()
response_content = json.loads(response.content)


df = spark.read.json(spark.sparkContext.parallelize([response_content]))

# Extract the bpi data into separate rows
result_df = df.select(
    explode(create_map(expr("'USD'"), col("bpi.USD"), expr("'GBP'"), col("bpi.GBP"), expr("'EUR'"), col("bpi.EUR"))).alias("currency", "data")
).select("currency", expr("data.*"))

# Display the DataFrame
result_df.show(truncate=False)

+--------+----+----------------------+----------+----------+-------+
|currency|code|description           |rate      |rate_float|symbol |
+--------+----+----------------------+----------+----------+-------+
|USD     |USD |United States Dollar  |47,851.831|47851.8307|&#36;  |
|GBP     |GBP |British Pound Sterling|37,910.422|37910.4215|&pound;|
|EUR     |EUR |Euro                  |44,401.953|44401.953 |&euro; |
+--------+----+----------------------+----------+----------+-------+

